# Importing The libraries 

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/anime-recommendations-database/rating.csv
/kaggle/input/anime-recommendations-database/anime.csv


# The data we have is very large and will take ridiculous amount to compute so we are going to take only the first 900k rows

In [2]:
ratings_in = pd.read_csv('../input/anime-recommendations-database/rating.csv')

movies_in = pd.read_csv('../input/anime-recommendations-database/anime.csv')
anime = movies_in[['anime_id','name']]
#Merging the dataframes 
ratings = pd.merge(anime, ratings_in)
ratings = ratings.head(900000)

In [3]:
ratings.head()

,anime_id,name,user_id,rating
0,32281,Kimi no Na wa.,99,5
1,32281,Kimi no Na wa.,152,10
2,32281,Kimi no Na wa.,244,10
3,32281,Kimi no Na wa.,271,10
4,32281,Kimi no Na wa.,278,-1


# Now we are going to make a user / anime rating matrix

In [4]:
userRatings = ratings.pivot_table(index=['user_id'],columns=['name'],values='rating')
userRatings.head()

name,Ano Hi Mita Hana no Namae wo Bokutachi wa Mada Shiranai.,Ansatsu Kyoushitsu (TV) 2nd Season,Aria The Origination,Ashita no Joe 2,Baccano!,Bakemono no Ko,Bakuman. 2nd Season,Bakuman. 3rd Season,Barakamon,Boku dake ga Inai Machi,...,Toki wo Kakeru Shoujo,Tonari no Totoro,Toradora!,Tsubasa: Tokyo Revelations,Uchuu Kyoudai,Uchuu Senkan Yamato 2199,Usagi Drop,Yojouhan Shinwa Taikei,Yuu☆Yuu☆Hakusho,Zoku Natsume Yuujinchou
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,7.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,8.0,NaN,...,8.0,NaN,3.0,NaN,8.0,NaN,7.0,NaN,7.0,NaN


# Now we check the  number of ratings so we can see what rows will be considered significant 

In [5]:
NumberOfRatings = ratings['name'].value_counts()
print (NumberOfRatings.median(),NumberOfRatings.mean(),NumberOfRatings.min())

4954.5 6818.181818181818 44


# We create a correlation matrix of each anime to all other animes . Yamete kudasai

In [6]:
corrMatrix = userRatings.corr()
corrMatrix.head()

name,Ano Hi Mita Hana no Namae wo Bokutachi wa Mada Shiranai.,Ansatsu Kyoushitsu (TV) 2nd Season,Aria The Origination,Ashita no Joe 2,Baccano!,Bakemono no Ko,Bakuman. 2nd Season,Bakuman. 3rd Season,Barakamon,Boku dake ga Inai Machi,...,Toki wo Kakeru Shoujo,Tonari no Totoro,Toradora!,Tsubasa: Tokyo Revelations,Uchuu Kyoudai,Uchuu Senkan Yamato 2199,Usagi Drop,Yojouhan Shinwa Taikei,Yuu☆Yuu☆Hakusho,Zoku Natsume Yuujinchou
name,,,,,,,,,,,,,,,,,,,,,
Ano Hi Mita Hana no Namae wo Bokutachi wa Mada Shiranai.,1.000000,0.651449,0.685131,0.603294,0.678061,0.651184,0.696485,0.689504,0.682491,0.628088,...,0.641588,0.608571,0.701503,0.595000,0.628627,0.673083,0.699009,0.628306,0.629335,0.683164
Ansatsu Kyoushitsu (TV) 2nd Season,0.651449,1.000000,0.785143,0.969608,0.584357,0.744269,0.732121,0.726696,0.717897,0.745899,...,0.626531,0.601430,0.613451,0.568282,0.700273,0.840423,0.685556,0.611609,0.604734,0.711399
Aria The Origination,0.685131,0.785143,1.000000,0.863659,0.762897,0.679785,0.746613,0.717509,0.714178,0.553049,...,0.645693,0.666018,0.683342,0.757199,0.777455,0.857967,0.759757,0.695769,0.631471,0.718045
Ashita no Joe 2,0.603294,0.969608,0.863659,1.000000,0.665621,0.943220,0.723271,0.636664,0.533745,0.764333,...,0.705392,0.585785,0.475689,0.865726,0.652512,0.933298,0.646788,0.912392,0.642679,0.853047
Baccano!,0.678061,0.584357,0.762897,0.665621,1.000000,0.603174,0.661751,0.673994,0.641402,0.548346,...,0.677751,0.655334,0.661081,0.653129,0.670482,0.778424,0.660521,0.678598,0.635210,0.697965


In [7]:
corrMatrix = userRatings.corr(method='pearson', min_periods=100) #the min rating freq was 44 which is low so we are going to discard any value under 100
corrMatrix.head()

name,Ano Hi Mita Hana no Namae wo Bokutachi wa Mada Shiranai.,Ansatsu Kyoushitsu (TV) 2nd Season,Aria The Origination,Ashita no Joe 2,Baccano!,Bakemono no Ko,Bakuman. 2nd Season,Bakuman. 3rd Season,Barakamon,Boku dake ga Inai Machi,...,Toki wo Kakeru Shoujo,Tonari no Totoro,Toradora!,Tsubasa: Tokyo Revelations,Uchuu Kyoudai,Uchuu Senkan Yamato 2199,Usagi Drop,Yojouhan Shinwa Taikei,Yuu☆Yuu☆Hakusho,Zoku Natsume Yuujinchou
name,,,,,,,,,,,,,,,,,,,,,
Ano Hi Mita Hana no Namae wo Bokutachi wa Mada Shiranai.,1.000000,0.651449,0.685131,NaN,0.678061,0.651184,0.696485,0.689504,0.682491,0.628088,...,0.641588,0.608571,0.701503,0.595000,0.628627,0.673083,0.699009,0.628306,0.629335,0.683164
Ansatsu Kyoushitsu (TV) 2nd Season,0.651449,1.000000,0.785143,NaN,0.584357,0.744269,0.732121,0.726696,0.717897,0.745899,...,0.626531,0.601430,0.613451,0.568282,0.700273,0.840423,0.685556,0.611609,0.604734,0.711399
Aria The Origination,0.685131,0.785143,1.000000,NaN,0.762897,NaN,0.746613,0.717509,0.714178,0.553049,...,0.645693,0.666018,0.683342,0.757199,0.777455,0.857967,0.759757,0.695769,0.631471,0.718045
Ashita no Joe 2,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Baccano!,0.678061,0.584357,0.762897,NaN,1.000000,0.603174,0.661751,0.673994,0.641402,0.548346,...,0.677751,0.655334,0.661081,0.653129,0.670482,0.778424,0.660521,0.678598,0.635210,0.697965


# We test the system by seeeing what user 3 have liked and what would he like 

In [8]:
myRatings = userRatings.loc[3].dropna()
myRatings

name
Ano Hi Mita Hana no Namae wo Bokutachi wa Mada Shiranai.    10.0
Boku dake ga Inai Machi                                     10.0
Death Note                                                  10.0
Fate/stay night: Unlimited Blade Works 2nd Season            9.0
Fullmetal Alchemist: Brotherhood                            10.0
Haikyuu!!                                                    9.0
Haikyuu!! Second Season                                      9.0
Hotarubi no Mori e                                           7.0
Kuroko no Basket                                            10.0
Kuroko no Basket 2nd Season                                 10.0
Kuroko no Basket 3rd Season                                 10.0
Magi: The Kingdom of Magic                                  10.0
No Game No Life                                              9.0
Noragami Aragoto                                            -1.0
One Punch Man                                               -1.0
Sen to Chihiro no Ka

In [9]:
simAnime = pd.Series()
for i in range(0, len(myRatings.index)):
    print ("Adding sims for " + myRatings.index[i] + "...")
    # Retrieve similar Animes the ones that user 3 rated
    sims = corrMatrix[myRatings.index[i]].dropna()
    # Now scale its similarity by how well the user 3  rated this anime
    sims = sims.map(lambda x: x * myRatings[i])
    # Add the score to the list of similarity candidates
    simAnime = simAnime.append(sims)
    
#Glance at our results so far:
print ("sorting...")
simAnime.sort_values(inplace = True, ascending = False)
print (simAnime.head(10))

Adding sims for Ano Hi Mita Hana no Namae wo Bokutachi wa Mada Shiranai....
Adding sims for Boku dake ga Inai Machi...
Adding sims for Death Note...
Adding sims for Fate/stay night: Unlimited Blade Works 2nd Season...
Adding sims for Fullmetal Alchemist: Brotherhood...
Adding sims for Haikyuu!!...
Adding sims for Haikyuu!! Second Season...
Adding sims for Hotarubi no Mori e...
Adding sims for Kuroko no Basket...
Adding sims for Kuroko no Basket 2nd Season...
Adding sims for Kuroko no Basket 3rd Season...
Adding sims for Magi: The Kingdom of Magic...
Adding sims for No Game No Life...
Adding sims for Noragami Aragoto...
Adding sims for One Punch Man...
Adding sims for Sen to Chihiro no Kamikakushi...
Adding sims for Shingeki no Kyojin...
Adding sims for Shokugeki no Souma...
Adding sims for Slam Dunk...
sorting...
Ano Hi Mita Hana no Namae wo Bokutachi wa Mada Shiranai.    10.0
Boku dake ga Inai Machi                                     10.0
Shokugeki no Souma                           

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


In [10]:
simAnime = simAnime.groupby(simAnime.index).sum()

# Abracadabra 🪄

In [11]:
simAnime.sort_values(inplace = True, ascending = False)
simAnime.head(10)

Hajime no Ippo: Rising                               118.887761
Uchuu Senkan Yamato 2199                             118.176351
Magi: The Kingdom of Magic                           117.136868
Uchuu Kyoudai                                        116.837602
Kuroko no Basket 3rd Season                          116.699351
Kuroko no Basket 2nd Season                          116.663414
Kingdom 2nd Season                                   116.210113
Haikyuu!!                                            115.498025
Fate/stay night: Unlimited Blade Works 2nd Season    114.999216
Kuroko no Basket                                     114.792207
dtype: float64